# 使用 Milvus 和 DeepSeek 构建 RAG——民法典案例

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [2]:
# 安装用于加载 .env 的依赖
%pip install -q python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 从 .env 加载环境变量（若存在），并检查密钥是否可用
from dotenv import load_dotenv
import os

# 自动加载项目根目录或当前目录中的 .env 文件
load_dotenv()

api_key = os.getenv("DEEPSEEK_API_KEY")
print("DEEPSEEK_API_KEY 已设置:", bool(api_key))
if not api_key:
    print("未读取到 DEEPSEEK_API_KEY，请在项目根目录创建 .env，内容为：\nDEEPSEEK_API_KEY=你的密钥\n或在终端导出后再从终端启动 Jupyter。")


DEEPSEEK_API_KEY 已设置: True


### 准备数据

In [4]:
from glob import glob

text_lines = []

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [5]:
len(text_lines)

30

### 准备 LLM 和 Embedding 模型

In [6]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [7]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

/opt/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


生成一个测试嵌入并打印其维度和前几个元素。

In [8]:
test_embedding = embedding_model.encode_queries(["为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.01968201  0.03749649 -0.01334841 -0.07770625 -0.00519627  0.00014536
 -0.00327914 -0.03993666 -0.03854968  0.04800905]


## 将数据加载到 Milvus

### 创建 Collection

In [9]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_mfd_demo.db")

collection_name = "my_mfd_rag_collection"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [11]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 526481.67it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [12]:
question = "债务人不按期还债，债权人可以做哪些事情?"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [13]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [14]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u4e8c\u3001\u6743\u5229\u8d28\u6743\n\n**\u7b2c\u56db\u767e\u56db\u5341\u4e5d\u6761** \u53ef\u4ee5\u51fa\u8d28\u7684\u6743\u5229\u5305\u62ec\uff1a\n\uff08\u4e00\uff09\u6c47\u7968\u3001\u672c\u7968\u3001\u652f\u7968\uff1b\n\uff08\u4e8c\uff09\u503a\u5238\u3001\u5b58\u6b3e\u5355\uff1b\n\uff08\u4e09\uff09\u4ed3\u5355\u3001\u63d0\u5355\uff1b\n\uff08\u56db\uff09\u53ef\u4ee5\u8f6c\u8ba9\u7684\u57fa\u91d1\u4efd\u989d\u3001\u80a1\u6743\uff1b\n\uff08\u4e94\uff09\u53ef\u4ee5\u8f6c\u8ba9\u7684\u6ce8\u518c\u5546\u6807\u4e13\u7528\u6743\u3001\u4e13\u5229\u6743\u3001\u8457\u4f5c\u6743\u7b49\u77e5\u8bc6\u4ea7\u6743\u4e2d\u7684\u8d22\u4ea7\u6743\uff1b\n\uff08\u516d\uff09\u5e94\u6536\u8d26\u6b3e\uff1b\n\uff08\u4e03\uff09\u6cd5\u5f8b\u3001\u884c\u653f\u6cd5\u89c4\u89c4\u5b9a\u53ef\u4ee5\u51fa\u8d28\u7684\u5176\u4ed6\u8d22\u4ea7\u6743\u5229\u3002\n\n**\u7b2c\u56db\u767e\u4e94\u5341\u6761** \u4ee5\u6c47\u7968\u3001\u672c\u7968\u3001\u652f\u7968\u3001\u503a\u5238\u3001\u5b58\u6b3e\u5355\u3

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [15]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [16]:
context

'二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意

In [17]:
question

'债务人不按期还债，债权人可以做哪些事情?'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [18]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [19]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [20]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，关于债务人不按期还债时债权人可以采取的行动，主要涉及合同权利转让的相关规定（第五百五十条）。债权人可以将合同的权利（即债权）全部或部分转让给第三人，但需注意以下例外情况：根据合同性质不得转让、按照当事人约定不得转让、或依照法律规定不得转让。此外，债权人转让权利时，应当通知债务人；未经通知，该转让对债务人不发生效力。

此信息基于合同变更和转让章节，未直接涉及债务违约时的其他救济措施（如请求损害赔偿等，但第五百五十六条提到合同变更不影响请求损害赔偿的权利，可能间接相关）。建议参考更全面的法律条款以获得完整答案。

<translated>
根据提供的上下文，关于债务人不按期还债时债权人可以采取的行动，主要涉及合同权利转让的相关规定（第五百五十条）。债权人可以将合同的权利（即债权）全部或部分转让给第三人，但需注意以下例外情况：根据合同性质不得转让、按照当事人约定不得转让、或依照法律规定不得转让。此外，债权人转让权利时，应当通知债务人；未经通知，该转让对债务人不发生效力。

此信息基于合同变更和转让章节，未直接涉及债务违约时的其他救济措施（如请求损害赔偿等，但第五百五十六条提到合同变更不影响请求损害赔偿的权利，可能间接相关）。建议参考更全面的法律条款以获得完整答案。
</translated>
